# Fluktuationen und Dissipation

Siehe Wiedemann/Ingold: *Numerische Physik mit Python*, Springer-Spektrum 2024, ISBN 978-3-662-69566-1

---

In diesem Jupyter-Notebook wird die Brown'sche Bewegung dadurch modelliert, dass ein Teilchen vielen Stößen mit leichteren Teilchen ausgesetzt wird. Dabei wird die Dynamik der leichteren Teilchen außer Acht gelassen und nur angenommen, dass die Stöße statistisch unabhängig voneinander stattfinden und die Geschwindigkeit der leichteren Teilchen vor dem Stoß normalverteilt ist.

## Importanweisungen

Zusätzlich zu bereits bekannten Importanweisungen wird hier der Namensraum des SciPy-Moduls `stats` importiert, aus dem wir eine Funktion zur Durchführung einer linearen Regression verwenden werden.

In [ ]:
from math import sqrt
import numpy as np
from scipy import stats
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt

plt.style.use("numphyspy.style")

## Zeitentwicklung des Ensembles

### Stoßzeitpunkte

Für statistisch voneinander unabhängige Stöße sind die Zeitintervalle zwischen aufeinanderfolgenden Stößen exponentiell verteilt. Dabei ist die mittlere Zeit zwischen zwei Stößen durch `dt_coll` gegeben. Die Größe des zweidimensionalen NumPy-Arrays `delta_t` ist durch die Zahl der Realisierung `n_ensemble` und die Zahl der Stöße `n_coll` bestimmt. Die Zeitpunkte `t` der einzelnen Stöße ergeben sich aus diesen Intervallen durch Bildung einer kumulativen Summe.

In [ ]:
def kick_times(rng, dt_coll, n_coll, n_ensemble):
    delta_t = rng.exponential(scale=dt_coll,
                              size=(n_ensemble, n_coll))
    t = np.cumsum(delta_t, axis=1)
    return t

### Geschwindigkeit des Brown'schen Teilchens

Die Geschwindigkeit der leichten Teilchen ist normalverteilt, wobei die Standardabweichung durch `v2_stdev` festgelegt ist. Ausgehend von der Geschwindigkeit `v_1_init` des Brown'schen Teilchens werden dessen Geschwindigkeiten nach den jeweiligen Stößen berechnet und mit Hilfe des zweidimensionalen NumPy-Arrays `v1` zurückgegeben.

In [ ]:
def velocity_of_heavy_particle(rng, v2_stdev, v_1_init, m2,
                               n_coll, n_ensemble):
    v2 = rng.normal(scale=v2_stdev,
                    size=(n_ensemble, n_coll))
    v1 = np.zeros((n_ensemble, n_coll+1))
    v1[:, 0] = v_1_init
    prefactor1 = (1-m2)/(1+m2)
    prefactor2 = 2*m2/(1+m2)
    for nc in range(n_coll):
        v1[:, nc+1] = (prefactor1*v1[:, nc]
                       + prefactor2*v2[:, nc])
    return v1

### Geschwindigkeit als Funktion der Zeit

Der zu betrachtende Zeitraum erstreckt sich bis zu dem Zeitpunkt `t_max`, an dem zum ersten Mal ein Teilchen des Ensembles seinen letzten Stoß erfährt. Dieser Zeitraum wird in Intervalle der Länge `dt` eingeteilt. Anschließend werden für jedes Teilchen des Ensembles die Stoßzeitpunkte den durch `t_vals` gegebenen Zeitintervallen zugeordnet. Damit ergibt sich ein zweidimensionales NumPy-Array `v_time`, das für alle Teilchen des Ensembles die Geschwindigkeiten des Brown'schen Teilchens am Ende des jeweiligen Zeitintervalls enthält.

In [ ]:
def velocity_time(t, v1, dt, n_ensemble):
    t_max = np.min(t[:, -1])
    t_vals = np.arange(0, t_max, dt)
    v_time = np.zeros((n_ensemble, t_vals.shape[0]))
    v_time[:, 0] = v1[:, 0]
    for ensemble in range(n_ensemble):
        time_idx = np.searchsorted(t[ensemble, :], t_vals)
        for nr, idx in enumerate(time_idx):
            v_time[ensemble, nr] = v1[ensemble, idx]
    return v_time

### Geschwindigkeit und Beschleunigung

Unter Verwendung der zuvor definierten Funktionen werden zunächst die Zeiten der Stöße und die Geschwindigkeit des Brown'schen Teilchens nach den Stößen berechnet und in `t` sowie `v1` gespeichert. Anschließend werden diese Daten zusammengeführt, um die Geschwindigkeit auf einem äquidistanten Zeitgitter zu erhalten. Durch Differenzbildung lässt sich daraus die Beschleunigung als Funktion der Zeit bestimmen.

In [ ]:
def time_development(v_1_init, dt_coll, m2, v2_stdev,
                     n_coll, n_ensemble, dt):
    rng = np.random.default_rng(123456)
    t = kick_times(rng, dt_coll, n_coll, n_ensemble)
    v1 = velocity_of_heavy_particle(rng, v2_stdev, v_1_init,
                                    m2, n_coll, n_ensemble)
    v_time = velocity_time(t, v1, dt, n_ensemble)
    a_time = np.diff(v_time, axis=1) / dt
    return v_time, a_time

### Statistische Eigenschaften

Durch Mittelung über das Ensemble erhält man die Mittelwerte von Geschwindigkeit und Beschleunigung, die Varianz der Geschwindigkeit sowie die zeitliche Entwicklung der Kovarianz zwischen Geschwindigkeit und Beschleunigung. Im letzten Teil werden die Daten von Geschwindigkeit und Beschleunigung so aufbereitet, dass der Zusammenhang der beiden Größen als Punktewolke dargestellt und einer linearen Regression unterworfen werden kann.

In [ ]:
def statistics(v_time, a_time):
    v_time = np.delete(v_time, -1, 1)
    v_mean = np.mean(v_time, axis=0)
    v_var = np.var(v_time, axis=0)
    a_mean = np.mean(a_time, axis=0)
    va_cov = np.mean(v_time*a_time, axis=0) - v_mean*a_mean

    v_all = np.ndarray.flatten(v_time)
    a_all = np.ndarray.flatten(a_time)
    lr_result = stats.linregress(v_all, a_all)
    return v_mean, v_var, va_cov, v_all, a_all, lr_result

## Implementierung der Bedienelemente und graphische Darstellung der Ergebnisse

Mit Hilfe der Schieberegler können die folgenden Parameter eingestellt werden:
- `v_1_init`: Anfangsgeschwindigkeit des Brown'schen Teilchens
- `dt_coll`: mittlere Zeit zwischen zwei Stößen
- `dt_relative`: Zeitintervalllänge bezogen auf die mittlere Zeit zwischen Kollisionen
- `log_n_coll`: Zehnerlogarithmus der Anzahl der Kollisionen
- `n_ensemble`: Anzahl der Teilchen im Ensemble

Neben der graphischen Darstellung der statistischen Ergebnisse werden auch Daten zur linearen Regression ausgegeben.

In [ ]:
widget_dict = {"v_1_init":
               widgets.FloatSlider(
                   value=1, min=-3, max=3, step=0.5,
                   description="$v_1(0)$"),
               "dt_coll":
               widgets.FloatSlider(
                   value=0.001, min=0.0001, max=0.01,
                   step=0.0001,
                   description=r"$\Delta t_\text{coll}$"),
               "dt_relative":
               widgets.IntSlider(
                   value=10, min=10, max=100, step=10,
                   description=r"$\Delta t/"
                               r"\Delta t_\text{coll}$"),
               "log_n_coll":
               widgets.IntSlider(
                   value=4, min=3, max=5, step=1,
                   description=r"$\log_{10}"
                               r"(n_\text{coll})$"),
               "n_ensemble":
               widgets.IntSlider(
                   value=1000, min=100, max=5000, step=100,
                   description="$N$"),
               }

@interact(**widget_dict)
def plot_result(v_1_init, dt_coll, dt_relative, log_n_coll,
                n_ensemble):
    dt = dt_coll * dt_relative
    m2 = dt_coll
    v2_stdev = sqrt(1 / m2)
    n_coll = 10**log_n_coll
    v_time, a_time = time_development(
        v_1_init, dt_coll, m2, v2_stdev, n_coll, n_ensemble,
        dt)
    (v_mean, v_var, va_cov, v_all, a_all,
        lr_result) = statistics(v_time, a_time)
    t = np.arange(v_mean.size)*dt

    print("Lineare Regression:")
    print()
    print(f"Steigung:          {lr_result.slope:8.4f}"
          f" ± {lr_result.stderr:5.3f}")
    print(f"y-Achsenabschnitt: {lr_result.intercept:8.4f}"
          f" ± {lr_result.intercept_stderr:5.3f}")
    print("Pearson-Korrelationskoeffizient:      "
          f"{lr_result.rvalue:9.6f}")
    print("Wahrscheinlichkeit der Nullhypothese: "
          f"{lr_result.pvalue:9.6f}")

    fig, axs = plt.subplots(2, 2)
    axs[0, 0].plot(t, v_mean, label=r"$\langle x \rangle$")
    axs[0, 0].set_xlabel("$t$")
    axs[0, 0].set_ylabel(r"$\langle v \rangle$")

    axs[0, 1].plot(t, v_var, label=r"$\langle x^2 \rangle$")
    axs[0, 1].set_xlabel("$t$")
    axs[0, 1].set_ylabel(r"$\sigma_v^2$")

    axs[1, 0].plot(t, va_cov, label=r"$\langle x \rangle$")
    axs[1, 0].set_xlabel("$t$")
    axs[1, 0].set_ylabel("$C(v,a)$")

    axs[1, 1].scatter(v_all, a_all, s=1)
    axs[1, 1].set_xlabel("$v$")
    axs[1, 1].set_ylabel("$a$")
    axs[1, 1].axline((0, lr_result.intercept),
                     slope=lr_result.slope, color="black")